# Banco do Brasil
Este código realiza web scraping de um site de agências bancárias do Banco do Brasil usando as bibliotecas Selenium, BeautifulSoup e Pandas. Ele obtém a URL redirecionada de uma página, faz o download do HTML, extrai informações específicas e as armazena em um DataFrame.



Aqui está o código comentado explicando cada etapa:

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

# Inicialização do webdriver do Chrome
driver = webdriver.Chrome(executable_path='C:\Webdriver\chromedriver.exe')

# Acessa a página principal
driver.get('https://bancosbrasil.com.br/agencias/banco-do-brasil-sa/goias/1')

# Obtém o HTML da página
html = driver.page_source

# Cria um objeto BeautifulSoup para fazer o parsing do HTML
soup = BeautifulSoup(html, 'html.parser')

# Seleciona os elementos que contêm os municípios
municipios = soup.select('#getCidades div[name][value]')

municipios_list = []
links_list = []

# Itera sobre os municípios e extrai o nome e o link de cada um
for municipio in municipios:
    nome = municipio['name']
    link = municipio.select_one('a')['href']
    municipios_list.append(nome)
    links_list.append(link)

# Encerra o webdriver
driver.quit()

# Listas para armazenar os dados extraídos
enderecos = []
bairros = []
telefones = []
links_coluna = []
informacoes_extras_coluna = []

# Itera sobre os links das agências
for link in links_list:
    # Inicializa um novo webdriver
    driver = webdriver.Chrome(executable_path='C:\Webdriver\chromedriver.exe')
    driver.get(link)

    # Obtém o HTML da página da agência
    html = driver.page_source

    # Cria um novo objeto BeautifulSoup para fazer o parsing do HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Encontra os links das agências específicas dentro da página da agência
    links = soup.find_all('a', href=lambda href: href.startswith('https://bancosbrasil.com.br/agencias/goias/'))

    # Itera sobre os links das agências específicas
    for link in links:
        driver.get(link['href'])

        # Obtém o HTML da página da agência específica
        html = driver.page_source

        # Cria um novo objeto BeautifulSoup para fazer o parsing do HTML
        soup = BeautifulSoup(html, 'html.parser')

        # Encontra as ocorrências das informações relevantes da agência
        ocorrencias = soup.find_all('div', class_='col-sm-12 p-1')

        # Verifica se há pelo menos duas ocorrências
        if len(ocorrencias) >= 2:
            segunda_ocorrencia = ocorrencias[1]

            card_body = segunda_ocorrencia.find('div', class_='card-body')
            endereco = ""
            bairro = ""

            # Extrai o endereço e o bairro da agência
            for p_tag in card_body.find_all('p'):
                if 'Endereço:' in p_tag.text:
                    endereco = p_tag.text.replace('Endereço:', '').strip()
                elif 'Bairro:' in p_tag.text:
                    bairro = p_tag.text.replace('Bairro:', '').strip()

            # Extrai o telefone da agência
            telefone_element = soup.find('p', style="cursor: pointer;")
            telefone = telefone_element.text.strip() if telefone_element else ""

            # Armazena os dados extraídos nas listas correspondentes
            enderecos.append(endereco)
            bairros.append(bairro)
            telefones.append(telefone)
            links_coluna.append(link['href'])
            informacoes_extras_coluna.append('Agência: ' + re.findall(r'\d+', link['href'])[-1])

    # Encerra o webdriver
    driver.quit()

# Cria um dicionário com os dados extraídos
dados = {
    'Endereço': enderecos,
    'Bairro': bairros,
    'Telefone': telefones,
    'Links': links_coluna,
    'Informações Extras': informacoes_extras_coluna,
}

# Cria um DataFrame com os dados adicionais
df_adicionais = pd.DataFrame(dados)

# Cria um DataFrame com os dados originais (municípios e links)
df_original = pd.DataFrame({
    'Municípios': municipios_list,
    'Links Bancos': links_list
})

# Concatena os DataFrames
df_completo = pd.concat([df_original, df_adicionais], axis=1)

# Remove as colunas 'Municípios' e 'Links Bancos'
df_completo = df_completo.drop(columns=['Municípios', 'Links Bancos'])

# Exibe o DataFrame completo
print(df_completo)


                Municípios                                       Links Bancos  \
0    Águas lindas de goiás  https://bancosbrasil.com.br/agencias/goias/agu...   
1    Águas lindas de goiás  https://bancosbrasil.com.br/agencias/goias/agu...   
2                 alexânia  https://bancosbrasil.com.br/agencias/goias/ale...   
3                 alexânia  https://bancosbrasil.com.br/agencias/goias/ale...   
4        alvorada do norte  https://bancosbrasil.com.br/agencias/goias/alv...   
..                     ...                                                ...   
295                    NaN                                                NaN   
296                    NaN                                                NaN   
297                    NaN                                                NaN   
298                    NaN                                                NaN   
299                    NaN                                                NaN   

                           

O código adiciona uma nova coluna chamada 'Municípios' ao DataFrame df_completo, que contém os nomes formatados dos municípios com base nos links existentes na coluna 'Links'. Em seguida, ele exibe a primeira linha do DataFrame atualizado.

In [ ]:
def format_municipio(link):
    # Extrai o nome do município do link
    municipio = link.split('/')[-2]
    # Substitui os traços por espaços e aplica o título ao nome do município
    municipio = municipio.replace('-', ' ').title()
    # Retorna o nome formatado do município
    return municipio

# Aplica a função 'format_municipio' à coluna 'Links' e adiciona os resultados à coluna 'Municípios'
df_completo['Municípios'] = df_completo['Links'].apply(format_municipio)

# Exibe a primeira linha do DataFrame atualizado
df_completo.head(1)

,Endereço,Bairro,Telefone,Links,Informações Extras,Municípios
0,"AV.JK,S/N QD.8,LOTE 18",JD BRASILIA,(61) 3613-8524,https://bancosbrasil.com.br/agencias/goias/agu...,Agência: 4590,Aguas Lindas Goias


O código remove as linhas duplicadas do DataFrame df_completo e, em seguida, salva o DataFrame atualizado em um arquivo CSV. O arquivo é salvo no diretório especificado com o nome "Banco do Brasil.csv" e as colunas do DataFrame são escritas sem incluir o índice.



In [ ]:
# Remove as linhas duplicadas do DataFrame
df_completo = df_completo.drop_duplicates()

# Salva o DataFrame atualizado em um arquivo CSV
df_completo.to_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Banco do Brasil.csv', index=False)


# Caixa Econômica Federal

O código usa a biblioteca Tabula para extrair dados de um arquivo PDF com a lista de todas as agências da Caixa Econômica Federal do país, para depois filtrar as agências de Goiás. Ele percorre várias páginas do PDF e armazena os dados em DataFrames separados. Os DataFrames são combinados em um único DataFrame chamado combined_df e são exibidos como saída.

In [ ]:
import tabula
import pandas as pd

file_path = r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\agencias_habilitadas_caixa_certificacao_digital.pdf'
pages = list(range(1, 49))

dfs = []

for page in pages:
    df = tabula.read_pdf(file_path, pages=page)[0]

    header = ["código", "unidade_caixa_no_unidade", "unidade_caixa_endereco", "unidade_caixa_bairro", "unidade_caixa_cidade", "UF", "telefone1", "Abertura", "Fechamento"]
    df.columns = header

    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

print(combined_df)


Got stderr: jun 06, 2023 3:18:00 PM org.apache.pdfbox.rendering.TTFGlyph2D getPathForGID
ADVERTÊNCIA: No glyph for code 0 (CID 0000) in font BCDGEE+Calibri



      código           unidade_caixa_no_unidade  \
0       4552                          XINGO, AL   
1       4025   JUSTICA ESTADUAL DE TERESINA, PI   
2       3950     JUSTICA FEDERAL RIO BRANCO, AC   
3       3706               SENADOR GUIOMARD, AC   
4       3423                     PORTO ACRE, AC   
...      ...                                ...   
3162    2370  JUSTICA FEDERAL FLORIANOPOLIS, SC   
3163    2358      JUSTICA FEDERAL JOINVILLE, SC   
3164    1271       JUSTICA FEDERAL LONDRINA, PR   
3165    1270  JUSTICA FEDERAL FOZ DO IGUACU, PR   
3166     650       JUSTICA FEDERAL CURITIBA, PR   

                                 unidade_caixa_endereco unidade_caixa_bairro  \
0     AVENIDA SAO FRANCISCO QUADRA B-17 - LOTE 11 NU...                XINGO   
1                        PRACA EDGARD NOGUEIRA  NUM S/N               CENTRO   
2     RODOVIA BR-364 RUA MINISTRO ILMAR N GALVAO , S...  DISTRITO INDUSTRIAL   
3           AVENIDA CASTELO BRANCO, 1270 - SLS. 06 A 11            

O código cria um novo DataFrame chamado new_df a partir das colunas selecionadas do DataFrame combined_df. Ele renomeia as colunas do new_df e adiciona colunas adicionais com valores específicos. O new_df é filtrado para incluir apenas as linhas com "UF" igual a "GO" e exibe o DataFrame final como saída.

In [ ]:
new_df = combined_df[["unidade_caixa_cidade", "unidade_caixa_endereco", "unidade_caixa_bairro", "telefone1", 'UF']].copy()

# Renomeia as colunas do DataFrame
new_df.rename(columns={"unidade_caixa_cidade": "Município", "unidade_caixa_endereco": "Endereço", "unidade_caixa_bairro": "Bairro", "telefone1": "Telefone"}, inplace=True)

# Adiciona colunas com valores específicos
new_df["Órgão"] = "Caixa Econômica Federal"
new_df["E-mail"] = ""
new_df["Horário de atendimento"] = combined_df.loc[:, "Abertura":"Fechamento"].apply(lambda x: " - ".join(x.dropna().astype(str)), axis=1)
new_df["Informação extra"] = "Agência: " + combined_df["código"].astype(str)

# Filtra o DataFrame para incluir apenas linhas com "UF" igual a "GO"
new_df = new_df[new_df["UF"] == "GO"]

# Seleciona as colunas desejadas e reordena o DataFrame
new_df = new_df[["Órgão", "Município", "Endereço", "Bairro", "Telefone", "E-mail", "Horário de atendimento", "Informação extra"]]

# Exibe o DataFrame final
print(new_df)

new_df.to_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Caixa Econômica Federal.csv', index=False)

                        Órgão                 Município  \
133   Caixa Econômica Federal      APARECIDA DE GOIANIA   
194   Caixa Econômica Federal                     GUAPO   
201   Caixa Econômica Federal  SANTA TEREZINHA DE GOIAS   
216   Caixa Econômica Federal                 ARAGARCAS   
264   Caixa Econômica Federal                   GOIANIA   
...                       ...                       ...   
3081  Caixa Econômica Federal                   GOIANIA   
3087  Caixa Econômica Federal       OURO VERDE DE GOIAS   
3088  Caixa Econômica Federal            ALTO HORIZONTE   
3089  Caixa Econômica Federal         SANTA FE DE GOIAS   
3141  Caixa Econômica Federal                       NaN   

                                               Endereço               Bairro  \
133         RUA DOUTOR ODILON DE AMORIM LOTE 01/03 Q 41  PARQUE VEIGA JARDIM   
194   RUA PADRE VICENTE LOTE 1A - COM RUA SAO SALVAD...               CENTRO   
201                AVENIDA DONA DITA - QUADRA A  LO

In [ ]:
new_df.to_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Caixa Econômica Federal.csv', index=False)

# Correios

Este código realiza web scraping de um site de agências dos Correios usando as bibliotecas Selenium, BeautifulSoup e Pandas. Ele obtém a URL redirecionada de uma página, faz o download do HTML, extrai informações específicas e as armazena em um DataFrame.

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd

# Caminho do webdriver do Chrome
webdriver_path = 'C:\Webdriver\chromedriver.exe'

# Configurações do Chrome Options para executar em modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")

# Inicialização do webdriver do Chrome
driver = webdriver.Chrome(executable_path=webdriver_path, options=chrome_options)

# URL do site dos Correios
url = 'https://mais.correios.com.br/app/index.php'
driver.get(url)

# Aguarda 3 segundos para garantir que a página seja carregada
time.sleep(3)

# Seleciona o estado "Goiás" no formulário
estado_select = Select(driver.find_element(By.NAME, 'cmbEstado'))
estado_select.select_by_visible_text('GOIÁS')

# Aguarda 3 segundos para o carregamento do município
time.sleep(3)

# Obtém a lista de municípios disponíveis no formulário
municipio_select = Select(driver.find_element(By.NAME, 'cmbMunicipio'))
municipios = [option.get_attribute('value') for option in municipio_select.options]

# Cria um DataFrame vazio para armazenar os dados
df = pd.DataFrame(columns=['Estado', 'Município', 'Bairro', 'Agência', 'Endereço'])

# Itera sobre os municípios
for municipio in municipios:
    # Seleciona o município atual no formulário
    municipio_select.select_by_value(municipio)

    # Aguarda 3 segundos para o carregamento do bairro
    time.sleep(3)

    # Obtém a lista de bairros disponíveis no formulário
    bairro_select = Select(driver.find_element(By.NAME, 'cmbBairro'))
    bairros = [option.get_attribute('value') for option in bairro_select.options]

    # Itera sobre os bairros
    for bairro in bairros:
        # Seleciona o bairro atual no formulário
        bairro_select.select_by_value(bairro)

        # Aguarda 3 segundos para a exibição dos dados da agência
        time.sleep(3)

        # Obtém o código HTML da página
        page_source = driver.page_source

        # Cria um objeto BeautifulSoup para fazer o parsing do HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Encontra os elementos de nome da agência e endereço
        agencia_elements = soup.find_all('span', class_='nome-agencia')
        endereco_elements = soup.find_all('span', class_='endereco-agencia')

        # Itera sobre os elementos e extrai as informações da agência
        for agencia_element, endereco_element in zip(agencia_elements, endereco_elements):
            agencia = agencia_element.text.strip()
            endereco = endereco_element.text.strip()

            # Adiciona as informações ao DataFrame
            df = df.append({
                'Estado': 'Goiás',
                'Município': municipio,
                'Bairro': bairro,
                'Agência': agencia,
                'Endereço': endereco
            }, ignore_index=True)

# Encerra o webdriver
driver.quit()

# Exibe o DataFrame com os dados das agências dos Correios em Goiás
print(df)


C:\Users\Renato Crozatti\AppData\Local\Temp\ipykernel_4780\475495972.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\Renato Crozatti\AppData\Local\Temp\ipykernel_4780\475495972.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\Renato Crozatti\AppData\Local\Temp\ipykernel_4780\475495972.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\Renato Crozatti\AppData\Local\Temp\ipykernel_4780\475495972.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\Renato Crozatti\AppData\Local\Temp\ipykernel_4780\475495972.py:74: FutureWarnin

    Estado        Município               Bairro             Agência  \
0    Goiás  ABADIA DE GOIAS                       AC ABADIA DE GOIAS   
1    Goiás  ABADIA DE GOIAS  SETOR SANTA BARBARA  AC ABADIA DE GOIAS   
2    Goiás        ABADIANIA                             AC ABADIANIA   
3    Goiás        ABADIANIA                           AGC PLANALMIRA   
4    Goiás        ABADIANIA                       AGC POSSE D ABADIA   
..     ...              ...                  ...                 ...   
671  Goiás         VILA BOA   JARDIM NOVA AURORA         AC VILA BOA   
672  Goiás    VILA PROPICIO                           AC DOIS IRMAOS   
673  Goiás    VILA PROPICIO                         AC VILA PROPICIO   
674  Goiás    VILA PROPICIO          DOIS IRMAOS      AC DOIS IRMAOS   
675  Goiás    VILA PROPICIO        SETOR CENTRAL    AC VILA PROPICIO   

                                      Endereço  
0                        AVENIDA COMERCIAL, SN  
1                        AVENIDA COME

In [ ]:
import numpy as np

# Cria uma cópia do DataFrame original
df_final = df

# Substitui os valores vazios (apenas espaços em branco) no campo 'Bairro' por NaN
df_final['Bairro'] = df_final['Bairro'].replace(r'^\s*$', np.nan, regex=True)

# Remove as linhas onde o campo 'Bairro' é NaN
df_final.dropna(subset=['Bairro'], inplace=True)

# Exibe o DataFrame final com as manipulações realizadas
df_final


,Estado,Município,Bairro,Agência,Endereço
1,Goiás,ABADIA DE GOIAS,SETOR SANTA BARBARA,AC ABADIA DE GOIAS,"AVENIDA COMERCIAL, SN"
5,Goiás,ABADIANIA,CENTRO,AC ABADIANIA,"AVENIDA GERALDO RODRIGUES DOS SANTOS, 327"
6,Goiás,ABADIANIA,CENTRO,AGC PLANALMIRA,"RUA QUARENTA E OITO, SN"
7,Goiás,ABADIANIA,CENTRO,AGC POSSE D ABADIA,"AV CORUMBA, SN"
9,Goiás,ACREUNA,SETOR CENTRAL,AC ACREUNA,"RUA ROSA GUIMARAES, 52 B"
...,...,...,...,...,...
667,Goiás,VIANOPOLIS,SETOR CENTRAL,AGC CARAIBA,AVENIDA PRINCIPAL
669,Goiás,VICENTINOPOLIS,SETOR CENTRAL,AC VICENTINOPOLIS,"RUA TANCREDO NEVES, 93"
671,Goiás,VILA BOA,JARDIM NOVA AURORA,AC VILA BOA,"RUA SETE DE SETEMBRO, 7"
674,Goiás,VILA PROPICIO,DOIS IRMAOS,AC DOIS IRMAOS,"RODOVIA BR 414, SN"


In [ ]:
# Obtém a contagem de ocorrências de cada valor único na coluna 'Município'
df_final['Município'].value_counts()

GOIANIA                 40
RIO VERDE                6
APARECIDA DE GOIANIA     6
ANAPOLIS                 6
FORMOSA                  4
                        ..
ISRAELANDIA              1
ITABERAI                 1
ITAGUARI                 1
ITAGUARU                 1
ABADIA DE GOIAS          1
Name: Município, Length: 245, dtype: int64

In [ ]:
# Atribui uma string fixa à coluna 'Horário de atendimento' do DataFrame - horário em que as agências dos correios funcionam em Goiás.
df_final['Horário de atendimento'] = 'Das 8h às 17h, com pausa das 12h às 14h'

# Exibe o DataFrame atualizado
df_final


,Estado,Município,Bairro,Agência,Endereço,Horário de atendimento
1,Goiás,ABADIA DE GOIAS,SETOR SANTA BARBARA,AC ABADIA DE GOIAS,"AVENIDA COMERCIAL, SN","Das 8h às 17h, com pausa das 12h às 14h"
5,Goiás,ABADIANIA,CENTRO,AC ABADIANIA,"AVENIDA GERALDO RODRIGUES DOS SANTOS, 327","Das 8h às 17h, com pausa das 12h às 14h"
6,Goiás,ABADIANIA,CENTRO,AGC PLANALMIRA,"RUA QUARENTA E OITO, SN","Das 8h às 17h, com pausa das 12h às 14h"
7,Goiás,ABADIANIA,CENTRO,AGC POSSE D ABADIA,"AV CORUMBA, SN","Das 8h às 17h, com pausa das 12h às 14h"
9,Goiás,ACREUNA,SETOR CENTRAL,AC ACREUNA,"RUA ROSA GUIMARAES, 52 B","Das 8h às 17h, com pausa das 12h às 14h"
...,...,...,...,...,...,...
667,Goiás,VIANOPOLIS,SETOR CENTRAL,AGC CARAIBA,AVENIDA PRINCIPAL,"Das 8h às 17h, com pausa das 12h às 14h"
669,Goiás,VICENTINOPOLIS,SETOR CENTRAL,AC VICENTINOPOLIS,"RUA TANCREDO NEVES, 93","Das 8h às 17h, com pausa das 12h às 14h"
671,Goiás,VILA BOA,JARDIM NOVA AURORA,AC VILA BOA,"RUA SETE DE SETEMBRO, 7","Das 8h às 17h, com pausa das 12h às 14h"
674,Goiás,VILA PROPICIO,DOIS IRMAOS,AC DOIS IRMAOS,"RODOVIA BR 414, SN","Das 8h às 17h, com pausa das 12h às 14h"


In [ ]:
# Salva o DataFrame df_final em um arquivo CSV
df_final.to_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Correios.csv', index=False)

In [ ]:
# Como o dataframe final sobre a presença federal em Goiás possui nomes que estão em minúsculo ou maiúsculo, o código abaixo transforma todos os nomes de municípios, bairros, endereços, órgãos e informações extras em título (primeira letra maiúscula e demais minuúsculas)

import pandas as pd

# Lê o arquivo CSV e armazena os dados em um DataFrame
df_goias = pd.read_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Anuári de dados - Presença federal em Goiás - Lista completa.csv')

# Aplica a função str.title() às colunas relevantes do DataFrame para capitalizar a primeira letra de cada palavra
df_goias['Município'] = df_goias['Município'].str.title()
df_goias['Bairro'] = df_goias['Bairro'].str.title()
df_goias['Endereço'] = df_goias['Endereço'].str.title()
df_goias['Informação extra'] = df_goias['Informação extra'].str.title()
df_goias['Órgão'] = df_goias['Órgão'].str.title()

# Salva o DataFrame atualizado em um novo arquivo CSV
df_goias.to_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Lista Final.csv', index=False)

# Judiciário

In [ ]:
import pandas as pd

# Lista contendo informações extras sobre órgãos do Judiciário em Goiás
dados_extras = [
    ['Justiça Federal', 'Formosa', 'Rua Itiquira, nº 1000, esq. com Rua Lindolfo Gonçalves, Setor Nordeste, CEP: 73807-145, Formosa-GO', '(61) 3631 - 4416 - R. 7311', '01vara.frm@trf1.jus.br', '', 'Subseção Judiciária de Formosa - Municípios Jurisdicionados: Água Fria de Goiás, Alto Paraíso de Goiás, Alvorada do Norte, Buritinópolis, Cabeceiras, Campos Belos, Cavalcante, Damianópolis, Divinópolis de Goiás, Flores de Goiás, Formosa, Guarani de Goiás, Iaciara, Mambaí, Monte Alegre de Goiás, Nova Roma, Planaltina, Posse, São Domingos, São João dAliança, Simolândia, Sítio dAbadia, Teresina de Goiás, Vila Boa.', 'Juiz Federal EDUARDO LUIZ ROCHA CUBAS', 'Portal TRF1: https://portal.trf1.jus.br/sjgo/institucional/enderecos-e-telefones/'],
    ['Justiça Federal', 'Itumbiara', 'Avenida João Paulo II, nº 185 (Térreo do Edifício do Fórum), Bairro Ernestina Borges de Andrade, CEP 75528-370, Itumbiara-GO', '(64) 2103-6412', '01vara.iub@trf1.jus.br', '', 'Subseção Judiciária de Itumbiara - Municípios Jurisdicionados: Água Limpa, Aloândia, Bom Jesus de Goiás, Buriti Alegre, Cachoeira Dourada, Goiatuba, Inaciolândia, Itumbiara, Joviânia, Marzagão, Morrinhos, Panamá, Vicentinópolis.', 'Juiz Federal FRANCISCO VIEIRA NETO', 'Portal TRF1: https://portal.trf1.jus.br/sjgo/institucional/enderecos-e-telefones/'],
    ['Justiça Federal', 'Jataí', 'Rua Nicolau Zaidem, nº 1135, Qd. 45 (antigo Fórum da cidade), Vila Fátima, Centro, CEP 75803-055, Jataí-GO', '(64) 2102-2116', '01gabju.jti@trf1.jus.br', '', 'Subseção Judiciária de Jataí - Municípios Jurisdicionados: Aparecida do Rio Doce, Aporé, Cachoeira Alta, Caçu, Caiapônia, Chapadão do Céu, Dovelândia, Itajá, Itarumã, Jataí, Lagoa Santa, Mineiros, Palestina de Goiás, Paranaiguara, Perolândia, Portelândia, Santa Rita do Araguaia, São Simão, Serranópolis.', 'Juiz Federal RAFAEL DE SOUSA BRANQUINHO E ASSIS', 'Portal TRF1: https://portal.trf1.jus.br/sjgo/institucional/enderecos-e-telefones/'],
    ['Justiça Federal', 'Luziânia', 'Rua Dr. João Teixeira, Quadra 73, Lote 21-A, nº 596, Edifício Iaci Amaral, Centro, CEP: 72.800-440, Luziânia/GO', '(61) 2104-3505', '01vara.lza@trf1.jus.br', '', 'Subseção Judiciária de Luziânia - Municípios Jurisdicionados: Cidade Ocidental, Cristalina, Luziânia, Mimoso de Goiás, Novo Gama, Padre Bernardo, Valparaíso de Goiás.', 'Juiz Federal TÁRSIS AUGUSTO DE SANTANA LIMA', 'Portal TRF1: https://portal.trf1.jus.br/sjgo/institucional/enderecos-e-telefones/'],
    ['Justiça Federal', 'Rio Verde', 'Avenida José Walter, nº 500, Quadra 49, Lotes 10/11, Setor Morada do Sol, Rio Verde/GO - CEP: 75908-740', '(64) 3211-8620', '01gabju.rvd@trf1.jus.br', '', 'Subseção Judiciária de Rio Verde - Municípios Jurisdicionados: Acreúna, Castelândia, Gouvelândia, Maurilândia, Montividiu, Porteirão, Quirinópolis, Rio Verde, Santa Helena de Goiás, Santo Antônio da Barra, Turvelândia.', 'Juiz Federal PAULO AUGUSTO MOREIRA LIMA', 'Portal TRF1: https://portal.trf1.jus.br/sjgo/institucional/enderecos-e-telefones/'],
    ['Justiça Federal', 'Uruaçu', 'Av. Tocantins, nº 17, Qd. 07, Lt.16, Setor Central, CEP: 76400-000, Uruaçu-GO', '(62) 3357-1070 - R. 7214', '01gabju.urc@trf1.jus.br', '', 'Subseção Judiciária de Uruaçu - Municípios Jurisdicionados: Alto Horizonte, Amaralina, Barro Alto, Bonópolis, Campinaçu, Campinorte, Campos Verdes, Carmo do Rio Verde, Ceres, Colinas do Sul, Crixás, Estrela do Norte, Formoso, Guarinos, Hidrolina, Ipiranga de Goiás, Itapaci, Mara Rosa, Minaçu, Montividiu do Norte, Mundo Novo, Mutunópolis, Niquelândia, Nova América, Nova Crixás, Nova Glória, Nova Iguaçu de Goiás, Novo Planalto, Pilar de Goiás, Porangatu, Rialma, Rubiataba, Santa Isabel, Santa Rita do Novo Destino, Santa Tereza de Goiás, Santa Terezinha de Goiás, São Luíz do Norte, São Miguel do Araguaia, Trombas, Uirapuru, Uruaçu.', 'Juiz Federal BRUNO TEIXEIRA DE CASTRO', 'Portal TRF1: https://portal.trf1.jus.br/sjgo/institucional/enderecos-e-telefones/']
]
judiciario_go = pd.DataFrame(dados_extras, columns=['Órgão', 'Município', 'Endereço', 'Telefone', 'E-mail', 'Horário de atendimento', 'Informação extra', 'Responsável', 'Fonte'])
judiciario_go

,Órgão,Município,Endereço,Telefone,E-mail,Horário de atendimento,Informação extra,Responsável,Fonte
0,Justiça Federal,Formosa,"Rua Itiquira, nº 1000, esq. com Rua Lindolfo G...",(61) 3631 - 4416 - R. 7311,01vara.frm@trf1.jus.br,,Subseção Judiciária de Formosa - Municípios Ju...,Juiz Federal EDUARDO LUIZ ROCHA CUBAS,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...
1,Justiça Federal,Itumbiara,"Avenida João Paulo II, nº 185 (Térreo do Edifí...",(64) 2103-6412,01vara.iub@trf1.jus.br,,Subseção Judiciária de Itumbiara - Municípios ...,Juiz Federal FRANCISCO VIEIRA NETO,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...
2,Justiça Federal,Jataí,"Rua Nicolau Zaidem, nº 1135, Qd. 45 (antigo Fó...",(64) 2102-2116,01gabju.jti@trf1.jus.br,,Subseção Judiciária de Jataí - Municípios Juri...,Juiz Federal RAFAEL DE SOUSA BRANQUINHO E ASSIS,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...
3,Justiça Federal,Luziânia,"Rua Dr. João Teixeira, Quadra 73, Lote 21-A, n...",(61) 2104-3505,01vara.lza@trf1.jus.br,,Subseção Judiciária de Luziânia - Municípios J...,Juiz Federal TÁRSIS AUGUSTO DE SANTANA LIMA,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...
4,Justiça Federal,Rio Verde,"Avenida José Walter, nº 500, Quadra 49, Lotes ...",(64) 3211-8620,01gabju.rvd@trf1.jus.br,,Subseção Judiciária de Rio Verde - Municípios ...,Juiz Federal PAULO AUGUSTO MOREIRA LIMA,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...
5,Justiça Federal,Uruaçu,"Av. Tocantins, nº 17, Qd. 07, Lt.16, Setor Cen...",(62) 3357-1070 - R. 7214,01gabju.urc@trf1.jus.br,,Subseção Judiciária de Uruaçu - Municípios Jur...,Juiz Federal BRUNO TEIXEIRA DE CASTRO,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...


# Versão final

O código concatena os DataFrames df_goias e judiciario_go em um novo DataFrame chamado df_final. Ele aplica a função str.title() à coluna 'Endereço' do DataFrame df_final para capitalizar a primeira letra de cada palavra. O DataFrame df_final é salvo em um arquivo CSV com o nome "Lista Final.csv" no diretório especificado, excluindo o índice, e é exibido como saída.

In [ ]:
import pandas as pd

# Concatena os DataFrames df_goias e judiciario_go em um novo DataFrame df_final
df_final = pd.concat([df_goias, judiciario_go])

# Aplica a função str.title() à coluna 'Endereço' do DataFrame df_final para capitalizar a primeira letra de cada palavra
df_final['Endereço'] = df_final['Endereço'].str.title()

# Salva o DataFrame df_final em um arquivo CSV
df_final.to_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Lista Final.csv', index=False)

# Exibe o DataFrame df_final
df_final


,Órgão,Município,Bairro,Endereço,Telefone,E-mail,Horário de atendimento,Informação extra,Responsável,Fonte,Email
0,Agência De Fomento De Goiás S/A - Goiásfomento,Goiânia,St. Central,"Av. Goiás, Nº 91 - St. Central, Goiânia - Go, ...",(62) 3216-4900,atendimento@goiasfomento.com,09h às 17h – segunda a sexta-feira,Braço Estadual Do Banco Central Do Brasil Foca...,NaN,Banco Central do Brasil: https://www.bcb.gov.b...,NaN
1,Agência Nacional De Telecomunicações - Anatel,Goiânia,St. Marista,"R. 13, 618 - St. Marista, Goiânia - Go, 74150-140",(62) 3236-9000,NaN,"Segunda a sexta-feira, das 8h às 12h e das 13h...",NaN,NaN,Anatel: https://www.gov.br/anatel/pt-br,NaN
2,Agência Nacional De Vigilância Sanitária - Anvisa,Goiânia,St. Sul,"R. 120, 304 - St. Sul, Goiânia - Go, 74085-450",(62) 3265-1588,NaN,"Segunda a sexta-feira, das 8h às 14h",NaN,NaN,Anvisa: https://www.gov.br/anvisa,NaN
3,Agência Nacional De Vigilância Sanitária - Anvisa,Goiânia,Santa Genoveva,"Alameda 4, S/N - Santa Genoveva",(62) 3265-1500,NaN,Emissão de certificação internacional de vacin...,Regional Do Aeroporto De Goiânia\nLocalização:...,NaN,Aeroporto de Goiânia: https://www.aeroportogoi...,NaN
4,Banco Do Brasil,Águas Lindas Goiás,Jd Brasilia,"Av.Jk,S/N Qd.8,Lote 18",(61) 3613-8524,NaN,10:00 - 16:00,Agência: 4590,NaN,Banco do Brasil e Bancos Brasil: https://banco...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1,Justiça Federal,Itumbiara,NaN,"Avenida João Paulo Ii, Nº 185 (Térreo Do Edifí...",(64) 2103-6412,01vara.iub@trf1.jus.br,,Subseção Judiciária de Itumbiara - Municípios ...,Juiz Federal FRANCISCO VIEIRA NETO,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...,NaN
2,Justiça Federal,Jataí,NaN,"Rua Nicolau Zaidem, Nº 1135, Qd. 45 (Antigo Fó...",(64) 2102-2116,01gabju.jti@trf1.jus.br,,Subseção Judiciária de Jataí - Municípios Juri...,Juiz Federal RAFAEL DE SOUSA BRANQUINHO E ASSIS,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...,NaN
3,Justiça Federal,Luziânia,NaN,"Rua Dr. João Teixeira, Quadra 73, Lote 21-A, N...",(61) 2104-3505,01vara.lza@trf1.jus.br,,Subseção Judiciária de Luziânia - Municípios J...,Juiz Federal TÁRSIS AUGUSTO DE SANTANA LIMA,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...,NaN
4,Justiça Federal,Rio Verde,NaN,"Avenida José Walter, Nº 500, Quadra 49, Lotes ...",(64) 3211-8620,01gabju.rvd@trf1.jus.br,,Subseção Judiciária de Rio Verde - Municípios ...,Juiz Federal PAULO AUGUSTO MOREIRA LIMA,Portal TRF1: https://portal.trf1.jus.br/sjgo/i...,NaN


# Contagem de órgãos federais

In [ ]:
# Realiza a contagem dos órgãos por município a partir do DataFrame df_final
orgaos_por_municipio = df_final.groupby('Município')['Órgão'].value_counts()

# Renomeia o resultado para 'Quantidade', reseta como DataFrame e salva em um arquivo CSV
df_contagem = orgaos_por_municipio.rename('Quantidade').reset_index()
df_contagem.to_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Final - Quantidade.csv', index=False)

# Exibe o DataFrame df_contagem
df_contagem


,Município,Órgão,Quantidade
0,Abadia De Goiás,Correios,1
1,Abadiania,Correios,3
2,Acreuna,Caixa Econômica Federal,1
3,Acreuna,Correios,1
4,Acreuna,Instituto Nacional Do Seguro Social (Inss),1
...,...,...,...
605,Vila Nova,Banco Do Brasil,1
606,Vila Propicio,Correios,2
607,Águas Lindas Goiás,Caixa Econômica Federal,2
608,Águas Lindas Goiás,Correios,2


In [ ]:
# Realiza a contagem dos órgãos federais a partir da coluna 'Órgão' do DataFrame df_final
orgaos = df_final['Órgão'].value_counts()

# Renomeia o resultado para 'Quantidade', reseta como DataFrame e atualiza o nome da coluna de índices
orgaos = orgaos.rename('Quantidade').reset_index()
orgaos = orgaos.rename(columns={'index': 'Órgãos federais'})

# Exibe o DataFrame orgaos
orgaos

,Órgãos federais,Quantidade
0,Correios,352
1,Banco Do Brasil,170
2,Caixa Econômica Federal,126
3,Instituto Nacional De Colonização E Reforma Ag...,58
4,Instituto Nacional Do Seguro Social (Inss),56
5,Ministério Do Trabalho E Emprego,15
6,Receita Federal,14
7,Exército Brasileiro,14
8,Justiça Federal,10
9,Consulado,7


In [ ]:
orgaos.to_csv(r'I:\Freelas\Anuário de Dados - Goiás\Presença federal\Final - Orgaos federais.csv', index=False)